In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score
import os

path = os.path.join(os.getcwd(), r"Redacted-Department _ CAD Reconciliation.csv")

df = pd.read_csv(path)

# Task A1: What is the range of data for the cases (dispatches) ?
# The data contains ID numbers, (dates and time) , shifts, and dispatch units

# Task A2: What % of data is missing, by each column?
#XREF ID                   0.000000%
#DISPATCH UNIT             0.000000%
#DISPATCH CREATED DATE     0.000000%
#INCIDENT NUMBER           0.000000%
#1ST UNIT ON SCENE        19.454545%
#ALARM DATE TIME           1.409091%
#CALL COMPLETE             1.409091%
#SHIFT                     3.136364%

missing_percent = df.isna().mean() * 100
print("Missing data % by column:\n", missing_percent)



# Task A3. What data issues are there (e.g., different formats) and how we can resolve them
# The first unit on scene is not recorded almost 20% of the time and the alarm date time, call complete time, and shift are occasionally not reported.
# We can resolve this by either dropping entries with missing data, or by replacing missing data with its mean, median, or mode.

# Task A4. Resolve data issues. Assign IDs. Pick a method for handling missing data and use consistently.
# Describe your data cleaning strategy, as appropriate. Do remainder of the tasks with data resolved. 
# I will be using the first stategy I mentioned above by deleting entries with missing data.

df_clean = df.dropna().copy()


# TASK B1. On an average, in how much time is a call (alarm) resolved from the time it is created to closed
df_clean["ALARM DATE TIME"] = pd.to_datetime(df_clean["ALARM DATE TIME"], format="%m/%d/%y %H:%M", errors="coerce")
df_clean["CALL COMPLETE"] = pd.to_datetime(df_clean["CALL COMPLETE"], format="%m/%d/%y %H:%M", errors="coerce")

df_clean = df_clean.dropna(subset=["ALARM DATE TIME", "CALL COMPLETE"])

df_clean["Resolution_Time"] = (df_clean["CALL COMPLETE"] - df_clean["ALARM DATE TIME"]).dt.total_seconds() / 60

avg_resolution = df_clean["Resolution_Time"].mean()
print(f"Average resolution time: {avg_resolution:.2f} minutes")

# 125722.06 minutes is equivalent to 87 Days, 6 Hours, 43 Minutes 

# TASK B2. How many fire units, on an average, are usually sent for a fire alarm
df_clean["Num_Units"] = df_clean["DISPATCH UNIT"].apply(lambda x: len(str(x).split(",")))
avg_units = df_clean["Num_Units"].mean()
print(f"Average number of fire units per alarm: {avg_units:.2f}")
#1.48 Fire Units


# TASK B3. Which shift is the busiest among A, B, C 
shift_counts = df_clean["SHIFT"].value_counts()
print("Number of alarms per shift:\n", shift_counts)

busiest_shift = shift_counts.idxmax()
print(f"\nThe busiest shift is: {busiest_shift}")
# Shift A is the busiest with 590 incidents 


# TASK B4. Create a matrix of number of file alarms organized by the day of week (x_axis) and hour of the day 
df_clean["DayOfWeek"] = df_clean["ALARM DATE TIME"].dt.day_name()
df_clean["Hour"] = df_clean["ALARM DATE TIME"].dt.hour

alarm_matrix = df_clean.pivot_table(
    index="Hour",
    columns="DayOfWeek",
    values="XREF ID",
    aggfunc="count",
    fill_value=0
)

week_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
alarm_matrix = alarm_matrix[week_order]

alarm_matrix['Total'] = alarm_matrix.sum(axis=1)
alarm_matrix.loc['Total'] = alarm_matrix.sum(axis=0)

print(alarm_matrix)

# Removes Totals
alarm_matrix_clustering = alarm_matrix.drop("Total", axis=0).drop("Total", axis=1)


# TASK C1. Cluster the data based on any two methods in sci-kit and report on their cluster quality. Which method performs better ? 
# I used a k-means silhouette to get a score of .4118 and dbscan to get a score of .2445. They both performed the strongest at two clusters.
# The k-means method performed better because its score was closest to 1.

# K-Means
kmeans = KMeans(n_clusters=2, random_state=42)
kmeans_labels = kmeans.fit_predict(alarm_matrix_clustering)

silhouette_kmeans = silhouette_score(alarm_matrix_clustering, kmeans_labels)
print(f"K-Means silhouette score: {silhouette_kmeans:.4f}")
#.4118


# DBSCAN
dbscan = DBSCAN(eps=10, min_samples=2)
dbscan_labels = dbscan.fit_predict(alarm_matrix_clustering)

silhouette_dbscan = silhouette_score(alarm_matrix_clustering, dbscan_labels)
print(f"DBSCAN silhouette score: {silhouette_dbscan:.4f}")
#.2445

# TASK C2. 





Missing data % by column:
 XREF ID                   0.000000
DISPATCH UNIT             0.000000
DISPATCH CREATED DATE     0.000000
INCIDENT NUMBER           0.000000
1ST UNIT ON SCENE        19.454545
ALARM DATE TIME           1.409091
CALL COMPLETE             1.409091
SHIFT                     3.136364
dtype: float64
Average resolution time: 125722.06 minutes
Average number of fire units per alarm: 1.48
Number of alarms per shift:
 SHIFT
A    593
C    570
B    557
Name: count, dtype: int64

The busiest shift is: A
DayOfWeek  Monday  Tuesday  Wednesday  Thursday  Friday  Saturday  Sunday  \
Hour                                                                        
0               7        6          4         1       2         7       4   
1               8        7          7         4       3         5       7   
2               4        3          2         2       3         7       7   
3               6        8          9         1      10         4       4   
4              